## ReMKiT1D input generator - staggered grids with a SOL-KiT style problem - isothermal

This example shows how staggered grids can be used in a setting which is comparable to a SOL-KiT fluid mode simulation with fixed temperature. Here linear extrapolation is used with Ti=Te=5eV.


The following are dependencies for this example

In [ ]:
import numpy as np
import xarray as xr
import holoviews as hv
import matplotlib.pyplot as plt
import matplotlib as mpl
from holoviews import opts
import panel as pn
import pickle

import sys
sys.path.append('../')
from RMK_support import RKWrapper ,Grid
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io
import RMK_support.dashboard_support as ds
import RMK_support.common_models as cm
import RMK_support.crm_support as crm # This is the main python module for crm construction


### Some useful constants

In [ ]:
elCharge = 1.60218e-19
elMass = 9.10938e-31
amu = 1.6605390666e-27 #atomic mass unit
ionMass = 2.014*amu # deuterium mass
epsilon0 = 8.854188e-12 #vacuum permittivity 
heavySpeciesMass = 2.014 #in amus

### Wrapper initialization

In [ ]:
rk = RKWrapper()

### Global parameters for IO files

In [ ]:
rk.jsonFilepath = "./config.json" # Default value
hdf5Filepath = "./RMKOutput/RMK_SK_comp_staggered_isothermal/" 
rk.setHDF5Path(hdf5Filepath)

### Setting options for external libraries used by ReMKiT1D

#### MPI


In [ ]:
numProcsX = 2 # Number of processes in x direction
numProcsH = 1 # Number of processes in harmonic 
numProcs = numProcsX * numProcsH
haloWidth = 1 # Halo width in cells

rk.setMPIData(numProcsX,numProcsH,haloWidth)

### Normalization

In [ ]:
rk.setNormDensity(1.0e19)
rk.setNormTemperature(10.0)
rk.setNormRefZ(1.0)

#Time normalization taken from already initialized config file with same input as above (this is a little inconvenient, but normalization should not change frequently enough to warrant anything more involved)
timeNorm = 0.72204953888999173E-7 # in s
# for convenience
tempNorm = rk.normalization["eVTemperature"] 
densNorm = rk.normalization["density"]

#Cross-section normalization taken from initialized config file
sigmaNorm = 0.73842563254919593E-18 # in m^{2}
lengthNorm = 0.13542325129584085E+0

### Grid initialization

In [ ]:
dx0 = 0.27
dxN = 0.0125
Nx = 128 
xGridWidths = np.geomspace(dx0,dxN,Nx)
L = sum(xGridWidths)
dv0 = 0.05
dvN = 0.4 
Nv = 80 
vGridWidths = np.geomspace(dv0,dvN,Nv)
lMax = 0
gridObj = Grid(xGridWidths,vGridWidths,lMax,interpretXGridAsWidths=True,interpretVGridAsWidths=True,isLengthInMeters=True)

dxNNorm = dxN/lengthNorm
dxNStagNorm = dxNNorm + xGridWidths[-2]/(2*lengthNorm)

In [ ]:
# Add the grid to the config file
rk.grid = gridObj

### Custom derivations

In [ ]:
# Diffusion coefficient derivation in 1D with neutral temperature Tn and with the cross section used being the low energy charge-exchange cross-seciton
# NOTE: SOL-KiT has a spurious sqrt(2) factor in the diffusion coefficient, so that is kept here for a consistent comparison
Tn = 3.0/tempNorm

diffusionDeriv = sc.simpleDerivation(np.sqrt(Tn)/2,[-1.0])

rk.addCustomDerivation("neutDiffD",diffusionDeriv)

rk.addCustomDerivation("identityDeriv",sc.simpleDerivation(1.0,[1.0]))
absDeriv = sc.multiplicativeDerivation("identityDeriv",[1],funcName="abs")
rk.addCustomDerivation("absDeriv",absDeriv)
rk.addCustomDerivation("square",sc.simpleDerivation(1.0,[2.0]))

### Custom extrapolation derivs

In [ ]:
rk.addCustomDerivation("linExtrapRight",sc.boundedExtrapolationDerivation(sc.linExtrapolation(),ignoreUpperBound=True))

rk.addCustomDerivation("linExtrapRightLB",sc.boundedExtrapolationDerivation(sc.linExtrapolation(),expectLowerBoundVar=True,ignoreUpperBound=True))

rk.addCustomDerivation("boundaryFlux",sc.multiplicativeDerivation("linExtrapRight",innerDerivationIndices=[1],outerDerivation="linExtrapRightLB",outerDerivationIndices=[2,3]))



### Handling particle species data

In [ ]:
rk.addSpecies("e",0,atomicA=elMass/amu,charge=-1.0,associatedVars=["ne","Ge","We"]) 
rk.addSpecies("D+",-1,atomicA=2.014,charge=1.0,associatedVars=["ni","Gi"])

# Set neutrals 
numNeutrals=1
neutralDensList = ["n"+str(i) for i in range(1,numNeutrals+1)] # List of neutral density names

for neutral in neutralDensList:
    rk.addSpecies("D"+neutral[1:],int(neutral[1:]),heavySpeciesMass,associatedVars=[neutral])

electronSpecies = rk.getSpecies("e")
ionSpecies = rk.getSpecies("D+")

#### Handling variables on staggered/dual grid

Variables can have the isOnDualGrid option set to true. Then, the variable values live on the dual grid (in 1D this is cell edges).


In [ ]:
n = np.ones(Nx)
T = 0.5*np.ones(Nx)
W = 3*n*T/2
# Set conserved variables in container

rk.addVarAndDual("ne",n,units='$10^{19} m^{-3}$',isCommunicated=True) #Units are not used by ReMKiT1D, but are useful to specify for later plotting
rk.addVarAndDual("ni",n,units='$10^{19} m^{-3}$',isCommunicated=True)
rk.addVarAndDual("Ge",primaryOnDualGrid=True,isCommunicated=True) # Ge_dual is evolved, and Ge is derived
rk.addVarAndDual("Gi",primaryOnDualGrid=True,isCommunicated=True)

# Temperatures
rk.addVarAndDual("Te",T,isDerived=True,units='$10eV$',isCommunicated=True)


# Set E field

rk.addVarAndDual("E",primaryOnDualGrid=True)

# Set derived fluid quantities

rk.addVarAndDual("ue",isDerived=True,primaryOnDualGrid=True,derivationRule=sc.derivationRule("flowSpeedFromFlux",["Ge_dual","ne_dual"]),isCommunicated=True)
rk.addVarAndDual("ui",isDerived=True,primaryOnDualGrid=True,derivationRule=sc.derivationRule("flowSpeedFromFlux",["Gi_dual","ni_dual"]),isCommunicated=True)
rk.addVar("cs",isDerived=True,derivationRule=sc.derivationRule("sonicSpeedD+",["Te","Te"]))

rk.addVar("cs_b",isDerived=True,isScalar=True,isCommunicated=True,hostScalarProcess=numProcs-numProcsH
          ,derivationRule=sc.derivationRule("linExtrapRight",["cs"]))

rk.addVar("n_b",isDerived=True,isScalar=True,isCommunicated=True,hostScalarProcess=numProcs-numProcsH
          ,derivationRule=sc.derivationRule("linExtrapRight",["ne"]))

rk.addVar("G_b",isDerived=True,isScalar=True,isCommunicated=True,hostScalarProcess=numProcs-numProcsH
          ,derivationRule=sc.derivationRule("boundaryFlux",["ni","ui","cs_b"]))

rk.addVar("u_b",isDerived=True,isScalar=True,isCommunicated=True,hostScalarProcess=numProcs-numProcsH
          ,derivationRule=sc.derivationRule("flowSpeedFromFlux",["G_b","n_b"]))

# Set scalar quantities 
rk.addVar("time",isScalar=True,isDerived=True)

# Set neutral densities

for neut in neutralDensList:
        rk.addVarAndDual(neut,units='$10^{19} m^{-3}$',isCommunicated=True)

# We need a distribution function to calculate rates from cross-sections built into the code
f = np.zeros([gridObj.numX(),gridObj.numH(),gridObj.numV()])
for i in range(gridObj.numX()):
    f[i,gridObj.getH(0)-1,:] = np.pi**(-1.5) * T[i] ** (-1.5) * n[i] * np.exp(-gridObj.vGrid**2/T[i])
rk.addVar("f",f,isDerived=True,isDistribution=True,derivationRule=sc.derivationRule("maxwellianDistribution",["Te","ne"]))



### Models 

### Density advection

In [ ]:
#Electron continuity advection

#Adding the model tag to tag list
modelTag = "continuity-ne"

#Initializing model using common models 
electronContModel = cm.staggeredAdvection(modelTag=modelTag, advectedVar="ne",
                                          fluxVar="Ge_dual", advectionSpeed="ue", lowerBoundVar="cs", rightOutflow=True)

rk.addModel(electronContModel.dict())

In [ ]:
#Ion continuity advection

#Adding the model tag to tag list
modelTag = "continuity-ni"

#Initializing model using common models
ionContModel = cm.staggeredAdvection(modelTag=modelTag, advectedVar="ni",
                                     fluxVar="Gi_dual", advectionSpeed="ui", lowerBoundVar="cs", rightOutflow=True)

rk.addModel(ionContModel.dict())

### Pressure gradient forces

In [ ]:
#Electron pressure grad

#Adding the model tag to tag list
modelTag = "pressureGrad-Ge"

#Initializing model
electronPressureGradModel = cm.staggeredPressureGrad(modelTag=modelTag,fluxVar="Ge_dual",densityVar="ne",temperatureVar="Te",speciesMass=elMass)

rk.addModel(electronPressureGradModel.dict())

In [ ]:
#Ion pressure grad

#Adding the model tag to tag list
modelTag = "pressureGrad-Gi"

#Initializing model
ionPressureGradModel = cm.staggeredPressureGrad(modelTag=modelTag,fluxVar="Gi_dual",densityVar="ni",temperatureVar="Te",speciesMass=ionMass)

rk.addModel(ionPressureGradModel.dict())

### Momentum advection

In [ ]:
#Electron momentum advection

#Adding the model tag to tag list
modelTag = "advection-Ge"

#Initializing model
electronMomAdvModel = cm.staggeredAdvection(modelTag=modelTag
                                        ,advectedVar="Ge_dual"
                                        ,fluxVar=""
                                        ,advectionSpeed="ue"
                                        ,staggeredAdvectionSpeed="ue_dual"
                                        ,lowerBoundVar="cs"
                                        ,rightOutflow=True,
                                        staggeredAdvectedVar=True)

rk.addModel(electronMomAdvModel.dict())

In [ ]:
#Ion momentum advection

#Adding the model tag to tag list
modelTag = "advection-Gi"

#Initializing model
ionMomAdvModel = cm.staggeredAdvection(modelTag=modelTag
                                        ,advectedVar="Gi_dual"
                                        ,fluxVar=""
                                        ,advectionSpeed="ui"
                                        ,staggeredAdvectionSpeed="ui_dual"
                                        ,lowerBoundVar="cs"
                                        ,rightOutflow=True,
                                        staggeredAdvectedVar=True)

rk.addModel(ionMomAdvModel.dict())

### Ampere-Maxwell term and Lorentz force

In [ ]:
#Ampere-Maxwell E field equation 
 
#Adding the model tag to tag list
modelTag = "ampereMaxwell"

#Initializing model
ampereMawellModel = cm.ampereMaxwell(modelTag=modelTag,
                                     eFieldName="E_dual",
                                     speciesFluxes=["Ge_dual","Gi_dual"],
                                     species=[electronSpecies,ionSpecies])

rk.addModel(ampereMawellModel.dict())

In [ ]:
#Lorentz force terms 
 
#Adding the model tag to tag list
modelTag = "lorentzForce"

#Initializing model
lorentzForceModel = cm.lorentzForces(modelTag=modelTag,
                                     eFieldName="E_dual",
                                     speciesFluxes=["Ge_dual","Gi_dual"],
                                     speciesDensities=["ne_dual","ni_dual"],
                                     species=[electronSpecies,ionSpecies])

rk.addModel(lorentzForceModel.dict())

### Neutral diffusion and recycling

In [ ]:
# Ground state diffusion and recyling

#Adding the model tag to tag list
modelTag = "neutDyn"

#Initializing model
neutDynModel = sc.CustomModel(modelTag=modelTag)

recConst = 1.0 # Recycling coef
normConstRec = sc.CustomNormConst(multConst=recConst,normNames=["speed","time","length"],normPowers=[1.0,1.0,-1.0])

sigmaCx = [3.0e-19, 2**4 * 1.0e-19, 3**4 * 7.0e-20] + [i**4 * 6.0e-20 for i in range(4,numNeutrals+1)]
normConstDiff = [sc.CustomNormConst(multConst = np.sqrt(elMass/ionMass) / (sigmaCx[i] / sigmaNorm), normNames=["density","length","crossSection"],normPowers=[-1.0,-1.0,-1.0]) for i in range(numNeutrals)]

# Diffusion term
for i in range(numNeutrals):
    evolvedVar = "n" + str(i+1)
    implicitVar = "n" + str(i+1)
    diffTerm = sc.GeneralMatrixTerm(evolvedVar,implicitVar,customNormConst=normConstDiff[i],stencilData=sc.diffusionStencil("neutDiffD",["ni_dual"],doNotInterpolate=True))
    neutDynModel.addTerm("neutralDiff"+str(i+1),diffTerm)

#Recycling term 

#Recycling term 
evolvedVar = "n1"
implicitVar = "ni"
recTerm = sc.GeneralMatrixTerm(evolvedVar,implicitVar,customNormConst=normConstRec,stencilData=sc.boundaryStencilDiv("ui","cs"),implicitGroups=[2])
neutDynModel.addTerm("recyclingTerm",recTerm)

rk.addModel(neutDynModel.dict())

### CX friction

In [ ]:
#Ion-neutral CX friction force terms 
 
#Adding the model tag to tag list
modelTag = "inFriction"

mbData = sc.VarlikeModelboundData()
mbData.addVariable("abs_ui",derivationRule=sc.derivationRule("absDeriv",["ui_dual"]))

#Initializing model
inFrictionModel = sc.CustomModel(modelTag=modelTag)

inFrictionModel.setModelboundData(mbData.dict())

# Use constant low-energy CX cross-sections
sigmaCx = [3.0e-19, 2**4 * 1.0e-19, 3**4 * 7.0e-20] + [i**4 * 6.0e-20 for i in range(4,numNeutrals+1)]

# Setting normalization constant calculation 
normConstCX = [sc.CustomNormConst(multConst=-sigmaCx[i]/sigmaNorm,normNames=["time","density","speed","crossSection"],normPowers=[1.0,1.0,1.0,1.0]) for i in range(numNeutrals)]

vDataIonCX = [sc.VarData(reqRowVars=["n" + str(i+1)+"_dual"],reqMBRowVars=["abs_ui"])  for i in range(numNeutrals)]

#Ion friction term 

evolvedVar = "Gi_dual"

implicitVar = "Gi_dual"

ionCXFriction = [sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstCX[i],varData=vDataIonCX[i],stencilData=sc.diagonalStencil()) for i in range(numNeutrals)]

for i in range(numNeutrals):
    inFrictionModel.addTerm("iFriction_cx"+str(i+1),ionCXFriction[i])

rk.addModel(inFrictionModel.dict())

### CRM density and energy evolution

In [ ]:
includedJanevTransitions = ["ion"] 
mbData = crm.ModelboundCRMData()
crm.addJanevTransitionsToCRMData(mbData,numNeutrals,tempNorm,"f","Te",detailedBalanceCSPriority=1,processes=includedJanevTransitions)


In [ ]:
#CRM model
 
#Adding the model tag to tag list
modelTag = "CRMmodel"

#Initializing model
crmModel = sc.CustomModel(modelTag=modelTag)

crmModel.setModelboundData(mbData.dict())

#Add ionization term generator for ions
ionInds,ionEnergies = mbData.getTransitionIndicesAndEnergies("JanevIon")
crmTermGeneratorIon = crm.termGeneratorCRM(implicitTermGroups=[2],evolvedSpeciesIDs=[-1],includedTransitionIndices=ionInds)
crmModel.addTermGenerator("crmTermGenIonIonization",crmTermGeneratorIon)

#Add all other terms for other particle species
crmTermGenerator = crm.termGeneratorCRM(evolvedSpeciesIDs=[0]+[i+1 for i in range(numNeutrals)])

crmModel.addTermGenerator("crmTermGen",crmTermGenerator)

rk.addModel(crmModel.dict())

### Integrator options

ReMKiT1D allows for highly customizable integrator options, with the default integrator being a composite integrator object containing Runge-Kutta explicit and Backwards Euler (with Picard iterations) implicit integrators. The properties of individual integrators can be modified and the integrators arranged in integration steps to produce more complicated integration schemes, such as Strang splitting. 

In this example only the Backwards Euler integrator is used, as set up below.

In [ ]:
integrator = sc.picardBDEIntegrator(absTol=100.0,convergenceVars=["ne","ni","Ge_dual","Gi_dual","n1"]) 

rk.addIntegrator("BE",integrator)

### Timestep control

Here the timestep is rescaled based on collisionality.

In [ ]:
initialTimestep=10.0

rk.setIntegratorGlobalData(3,2,initialTimestep) 

timestepControllerOptions = sc.scalingTimestepController(["ne","Te"],[-1.0,1.5])

rk.setTimestepController(timestepControllerOptions)

### Controlling integration steps

As mentioned above, ReMKiT1D alows for composing integrators in a sequence using integration steps. 

This example uses the simplest behaviour - a single step integration

In [ ]:
bdeStep = sc.IntegrationStep("BE",defaultEvaluateGroups=[1,2,3],defaultUpdateModelData=True,defaultUpdateGroups=[1,2,3])

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("StepBDE",bdeStep.dict())

### Time loop options

The main part of ReMKiT1D is the time loop, where the variables are advanced through time by repeatedly calling the integrators defined above. The following shows a way to set timeloop options based on a time target:

In [ ]:
rk.setTimeTargetTimestepping(160000.0)
rk.setMinimumIntervalOutput(8000.0)
rk.setRestartOptions(True, False, 1000) #Change to True when restarting

In [ ]:
terms = rk.getTermsThatEvolveVar("ne")

for pair in terms:
    model,term=pair
    rk.addVar(model+term,isDerived=True)
    rk.addManipulator(model+term,sc.termEvaluatorManipulator([pair],model+term))

In [ ]:
terms = rk.getTermsThatEvolveVar("Ge_dual")

for pair in terms:
    model,term=pair
    rk.addVar(model+term,isDerived=True)
    rk.addManipulator(model+term,sc.termEvaluatorManipulator([pair],model+term))

In [ ]:
terms = rk.getTermsThatEvolveVar("Gi_dual")

for pair in terms:
    model,term=pair
    rk.addVar(model+term,isDerived=True)
    rk.addManipulator(model+term,sc.termEvaluatorManipulator([pair],model+term))

In [ ]:
terms = rk.getTermsThatEvolveVar("n1")

for pair in terms:
    model,term=pair
    rk.addVar(model+term,isDerived=True)
    rk.addManipulator(model+term,sc.termEvaluatorManipulator([pair],model+term))

In [ ]:
terms = rk.getTermsThatEvolveVar("E_dual")

for pair in terms:
    model,term=pair
    rk.addVar(model+term,isDerived=True)
    rk.addManipulator(model+term,sc.termEvaluatorManipulator([pair],model+term))

In [ ]:
rk.addVar("ionsource",isDerived=True)
rk.addManipulator("ionsource",sc.groupEvaluatorManipulator("CRMmodel",2,"ionsource"))

### Write config file

In [ ]:
rk.writeConfigFile()

### Data analysis

In [ ]:
numFiles = 20

In [ ]:
loadFilenames = [hdf5Filepath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]

In [ ]:
loadedData = io.loadFromHDF5(rk.varCont,filepaths=loadFilenames,varsToIgnore=["ionGamma"])
loadedData

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=100,dpi=150)

#### Explore data using basic dashboard

In [ ]:
pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData,gridObj)

dashboard.fluid2Comparison().show()
